<a href="https://colab.research.google.com/github/Degananda264/Natural-Language-Processing-/blob/master/Predicting_E_Commerce_Product_Recommendation_Ratings_from_Reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report

In [17]:
df=pd.read_csv("Reviews.csv")

In [18]:
df.shape

(23486, 11)

In [19]:
df.isnull().sum()

Unnamed: 0                    0
Clothing ID                   0
Age                           0
Title                      3810
Review Text                 845
Rating                        0
Recommended IND               0
Positive Feedback Count       0
Division Name                14
Department Name              14
Class Name                   14
dtype: int64

In [20]:
df.head(3)

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses


In [21]:
df['Title']=df['Title'].fillna(' ')


In [22]:
df.head(4)

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants


In [24]:
df['Review'] = (df['Title'].map(str) +' '+ df['Review Text'])
df['Rating'] = [1 if rating > 3 else 0 for rating in df['Rating']]
df = df[['Review', 'Rating']]
df.head()

,Review,Rating
0,Absolutely wonderful - silky and sexy and co...,1
1,Love this dress! it's sooo pretty. i happe...,1
2,Some major design flaws I had such high hopes ...,0
3,"My favorite buy! I love, love, love this jumps...",1
4,Flattering shirt This shirt is very flattering...,1


In [25]:
df['Review'].isnull().sum()

845

In [26]:
df=df.dropna()

In [27]:
df['Review'].isnull().sum()

0

In [28]:
df.shape

(22641, 2)

In [29]:
df['Rating'].value_counts()

1    17448
0     5193
Name: Rating, dtype: int64

In [30]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df[['Review']], df['Rating'], random_state=42)
X_train.shape, X_test.shape

((16980, 1), (5661, 1))

In [31]:
from collections import Counter
Counter(y_train), Counter(y_test)

(Counter({0: 3890, 1: 13090}), Counter({0: 1303, 1: 4358}))

**Experiment 1: Basic NLP Count based Features**

In [32]:
import string

X_train['char_count'] = X_train['Review'].apply(len)
X_train['word_count'] = X_train['Review'].apply(lambda x: len(x.split()))
X_train['word_density'] = X_train['char_count'] / (X_train['word_count']+1)
X_train['punctuation_count'] = X_train['Review'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation))) 
X_train['title_word_count'] = X_train['Review'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
X_train['upper_case_word_count'] = X_train['Review'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))


X_test['char_count'] = X_test['Review'].apply(len)
X_test['word_count'] = X_test['Review'].apply(lambda x: len(x.split()))
X_test['word_density'] = X_test['char_count'] / (X_test['word_count']+1)
X_test['punctuation_count'] = X_test['Review'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation))) 
X_test['title_word_count'] = X_test['Review'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
X_test['upper_case_word_count'] = X_test['Review'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))

In [33]:
X_train.head(4)

,Review,char_count,word_count,word_density,punctuation_count,title_word_count,upper_case_word_count
1496,I wanted to like this... I wanted to like this...,525,104,5.000000,19,2,2
5205,Beautiful blouse Bought this for my daughter i...,203,35,5.638889,10,2,0
12786,I saw this dress online and thought it looke...,506,104,4.819048,24,1,1
20114,Summer love I wish it was a little bigger.i ke...,90,17,5.000000,3,2,1


**Logitic Regression Model**

In [34]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression(C=1,random_state=42,solver='liblinear')

In [35]:
lr.fit(X_train.drop(['Review'],axis=1),y_train)
predictions=lr.predict(X_test.drop(['Review'],axis=1))

In [36]:
print(classification_report(y_test, predictions))
pd.DataFrame(confusion_matrix(y_test, predictions))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1303
           1       0.77      1.00      0.87      4358

    accuracy                           0.77      5661
   macro avg       0.38      0.50      0.43      5661
weighted avg       0.59      0.77      0.67      5661



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,0,1
0,0,1303
1,0,4358


**Using Polarity and Subjectivity from TextBlob**

In [38]:
import textblob

textblob.TextBlob('this is perfect fitted jeans').sentiment

Sentiment(polarity=1.0, subjectivity=1.0)

In [39]:
import textblob

textblob.TextBlob('This is worst jeans!').sentiment

Sentiment(polarity=-1.0, subjectivity=1.0)

**Experiment 2: Features from Sentiment Analysis**

In [40]:
X_train_snt=X_train['Review'].apply(lambda row:textblob.TextBlob(row).sentiment)
X_train['Polarity']=[obj.polarity for obj in X_train_snt.values]
X_train['Subjectivity']=[obj.subjectivity for obj in X_train_snt.values]

In [42]:
X_test_snt=X_test['Review'].apply(lambda row:textblob.TextBlob(row).sentiment)
X_test['Polarity']=[obj.polarity for obj in X_test_snt.values]
X_test['Subjectivity']=[obj.subjectivity for obj in X_test_snt.values]

In [43]:
X_train.head()

,Review,char_count,word_count,word_density,punctuation_count,title_word_count,upper_case_word_count,Polarity,Subjectivity
1496,I wanted to like this... I wanted to like this...,525,104,5.000000,19,2,2,0.186538,0.458761
5205,Beautiful blouse Bought this for my daughter i...,203,35,5.638889,10,2,0,0.625000,0.825000
12786,I saw this dress online and thought it looke...,506,104,4.819048,24,1,1,0.237679,0.592857
20114,Summer love I wish it was a little bigger.i ke...,90,17,5.000000,3,2,1,0.437500,0.700000
19068,So pretty! I love this top! nice lightweight m...,159,31,4.968750,5,2,1,0.298148,0.624074


In [44]:
X_test.head(2)

,Review,char_count,word_count,word_density,punctuation_count,title_word_count,upper_case_word_count,Polarity,Subjectivity
13365,Stunning! This sweater is so beautiful on. it ...,384,73,5.189189,21,2,0,0.400000,0.702273
19834,This piece is almost what i want... i tried ...,502,101,4.921569,14,1,0,0.217985,0.466327


**Logistic Regression Model Training & Evaluation**

In [45]:
lr.fit(X_train.drop(['Review'],axis=1),y_train)
predictions=lr.predict(X_test.drop(['Review'],axis=1))

In [46]:
print(classification_report(y_test, predictions))
pd.DataFrame(confusion_matrix(y_test, predictions))

              precision    recall  f1-score   support

           0       0.69      0.25      0.37      1303
           1       0.81      0.97      0.88      4358

    accuracy                           0.80      5661
   macro avg       0.75      0.61      0.63      5661
weighted avg       0.78      0.80      0.76      5661



,0,1
0,328,975
1,147,4211


**Text Pre-processing and Wrangling of Reviews**

In [47]:
!pip install contractions
!pip install textsearch
!pip install tqdm
import nltk
nltk.download('punkt')
nltk.download('stopwords')

     |████████████████████████████████| 245kB 3.8MB/s 
     |████████████████████████████████| 317kB 18.2MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81707 sha256=4cdd3a404d016b3341c0a81cb57c772e1b02a59fb4ba6b714e6026f8f6231ef7
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [48]:
import nltk
import contractions
import re

# remove some stopwords to capture negation in n-grams if possible
stop_words = nltk.corpus.stopwords.words('english')
stop_words.remove('no')
stop_words.remove('not')
stop_words.remove('but')

# load up a simple porter stemmer - nothing fancy
ps = nltk.porter.PorterStemmer()

def simple_text_preprocessor(document): 
    # lower case
    document = str(document).lower()
    
    # expand contractions
    document = contractions.fix(document)
    
    # remove unnecessary characters
    document = re.sub(r'[^a-zA-Z]',r' ', document)
    document = re.sub(r'nbsp', r'', document)
    document = re.sub(' +', ' ', document)
    
    # simple porter stemming
    document = ' '.join([ps.stem(word) for word in document.split()])
    
    # stopwords removal
    document = ' '.join([word for word in document.split() if word not in stop_words])
    
    return document

stp = np.vectorize(simple_text_preprocessor)

In [49]:
X_train['Clean Review'] = stp(X_train['Review'].values)
X_test['Clean Review'] = stp(X_test['Review'].values)

X_train.head()

,Review,char_count,word_count,word_density,punctuation_count,title_word_count,upper_case_word_count,Polarity,Subjectivity,Clean Review
1496,I wanted to like this... I wanted to like this...,525,104,5.000000,19,2,2,0.186538,0.458761,want like thi want like thi top badli badli fa...
5205,Beautiful blouse Bought this for my daughter i...,203,35,5.638889,10,2,0,0.625000,0.825000,beauti blous bought thi daughter law birthday ...
12786,I saw this dress online and thought it looke...,506,104,4.819048,24,1,1,0.237679,0.592857,saw thi dress onlin thought look ok happen see...
20114,Summer love I wish it was a little bigger.i ke...,90,17,5.000000,3,2,1,0.437500,0.700000,summer love wish wa littl bigger kept thi wond...
19068,So pretty! I love this top! nice lightweight m...,159,31,4.968750,5,2,1,0.298148,0.624074,pretti love thi top nice lightweight materi bu...


**Extracting out the structured features from previous experiments**

In [50]:
X_train_metadata = X_train.drop(['Review', 'Clean Review'], axis=1).reset_index(drop=True)
X_test_metadata = X_test.drop(['Review', 'Clean Review'], axis=1).reset_index(drop=True)

X_train_metadata.head()

,char_count,word_count,word_density,punctuation_count,title_word_count,upper_case_word_count,Polarity,Subjectivity
0,525,104,5.000000,19,2,2,0.186538,0.458761
1,203,35,5.638889,10,2,0,0.625000,0.825000
2,506,104,4.819048,24,1,1,0.237679,0.592857
3,90,17,5.000000,3,2,1,0.437500,0.700000
4,159,31,4.968750,5,2,1,0.298148,0.624074


**Experiment 3: Adding Bag of Words based Features - 1-grams for Reviews**

In [51]:

from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(min_df=0.0, max_df=1.0, ngram_range=(1, 1))
X_traincv = cv.fit_transform(X_train['Clean Review']).toarray()
X_traincv = pd.DataFrame(X_traincv, columns=cv.get_feature_names())

X_testcv = cv.transform(X_test['Clean Review']).toarray()
X_testcv = pd.DataFrame(X_testcv, columns=cv.get_feature_names())
X_traincv.head()

,aa,aaaaaaamaz,aaaaandidon,aaaaannnnnnd,aaaahmaz,aaah,ab,abbey,abbi,abck,abdomen,abercrombi,abil,abject,abl,abnorm,abo,abolut,abou,abov,abroad,abruptli,abso,absolut,absoluti,absolutley,absolutli,absorb,abstract,absurd,absurdli,abt,abund,abut,ac,accent,accentu,accentuatea,accept,acceptabl,...,yoga,yogi,yogini,yoke,yolk,yoo,yore,york,yoself,young,younger,youth,youthful,yr,yuck,yucki,yuk,yum,yumi,yummi,yummiest,yummysweat,yup,zag,zara,zermatt,zero,zig,zigzag,zillion,zing,zip,ziploc,zipper,zipperi,zoe,zone,zooland,zoom,zuma
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [52]:
X_train_comb = pd.concat([X_train_metadata, X_traincv], axis=1)
X_test_comb = pd.concat([X_test_metadata, X_testcv], axis=1)

X_train_comb.head()

,char_count,word_count,word_density,punctuation_count,title_word_count,upper_case_word_count,Polarity,Subjectivity,aa,aaaaaaamaz,aaaaandidon,aaaaannnnnnd,aaaahmaz,aaah,ab,abbey,abbi,abck,abdomen,abercrombi,abil,abject,abl,abnorm,abo,abolut,abou,abov,abroad,abruptli,abso,absolut,absoluti,absolutley,absolutli,absorb,abstract,absurd,absurdli,abt,...,yoga,yogi,yogini,yoke,yolk,yoo,yore,york,yoself,young,younger,youth,youthful,yr,yuck,yucki,yuk,yum,yumi,yummi,yummiest,yummysweat,yup,zag,zara,zermatt,zero,zig,zigzag,zillion,zing,zip,ziploc,zipper,zipperi,zoe,zone,zooland,zoom,zuma
0,525,104,5.000000,19,2,2,0.186538,0.458761,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,203,35,5.638889,10,2,0,0.625000,0.825000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,506,104,4.819048,24,1,1,0.237679,0.592857,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,90,17,5.000000,3,2,1,0.437500,0.700000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,159,31,4.968750,5,2,1,0.298148,0.624074,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


**Logistic Regression Training & Evaluation**

In [53]:
lr.fit(X_train_comb, y_train)
predictions = lr.predict(X_test_comb)

print(classification_report(y_test, predictions))
pd.DataFrame(confusion_matrix(y_test, predictions))


              precision    recall  f1-score   support

           0       0.78      0.71      0.75      1303
           1       0.92      0.94      0.93      4358

    accuracy                           0.89      5661
   macro avg       0.85      0.83      0.84      5661
weighted avg       0.89      0.89      0.89      5661



,0,1
0,928,375
1,260,4098
